<a href="https://colab.research.google.com/github/romulokps/testesPCR/blob/main/resultadoTesteNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import csv
from datetime import datetime
from scipy import stats
import numpy as np

caminho = '/content/drive/My Drive/dados/estados 2020-11-07/'
caminho = '/content/drive/My Drive/dados/estados 2020-11-13/'
caminho = '/content/drive/My Drive/dados/estados 2020-10-21/'
caminho = '/content/drive/My Drive/dados/estados 2020-11-15/'
pop = pd.read_csv('https://raw.githubusercontent.com/romulokps/testesPCR/main/populacaoBR.csv')
pop = pop.sort_values(by = 'pop', ascending=False).head(25)
#pop = pop.sort_values(by = 'pop', ascending=False).tail(2)
#pop = pop.sort_values(by = 'pop', ascending=False).head(1)
pop = pop.sort_values(by = 'estado', ascending=False)
#pop = pop.tail(20)
#pop = pop.sort_values(by = 'pop', ascending=False).head(6)
pop = pop.reset_index()
for i in pop.index:
    if ((pop.iloc[i]['estado'] != pop.iloc[i-1]['estado']) ):
     df = pd.read_csv(caminho + pop.iloc[i]['estado'], low_memory = False)
     #df = pd.read_csv(caminho + pop.iloc[i]['estado'], low_memory = False, encoding="latin-1", sep=";", error_bad_lines=False)
    fe = df  
    #fe = fe[fe['tipoTeste'] == 'RT-PCR']
    #if (ibge != 0):
    fe = fe[fe['municipioNotificacaoIBGE'] == pop.iloc[i]['ibgeID']]
    #fe = fe[fe['municipioNotificacaoIBGE'] == str(pop.iloc[i]['ibgeID'])] # TIRAR ESSA LINHA SE FOR DADOS DO ES
    estado = pop.iloc[i]['cidade']
    #else:
      #estado = pop.iloc[i]['estadoNome']
    fe = fe[fe['resultadoTeste'].isnull()]
    
    #fe = fe[fe['resultadoTeste'] != 'Inconclusivo ou Indeterminado']
    fe = fe[['dataTeste', 'tipoTeste','estado']]
    fe['dataTeste'] = pd.to_datetime(fe['dataTeste']).dt.date
    fe = fe[fe['dataTeste'] > datetime.strptime('2020-04-01', '%Y-%m-%d').date()]
    #fe = fe.sort_values(by= ['dataTeste'], ascending=False)
    fe = fe.replace(np.nan, "nan")
    fe = pd.get_dummies(fe, columns=['tipoTeste'])
    fe = fe.groupby('dataTeste').sum()

    #tira os outliers
    z_scores = stats.zscore(fe)
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 13).all(axis=1)
    fe = fe[filtered_entries]

    #total = (fe['resultadoTeste_Positivo'] + fe['resultadoTeste_Negativo'] +  fe['resultadoTeste_nan'] + fe['resultadoTeste_Inconclusivo ou Indeterminado']) 
    #taxa = (fe['resultadoTeste_Positivo']/total).rolling(min_periods=1, window=janela).sum()/janela
    #taxai = (fe['resultadoTeste_Positivo']/(total - fe['resultadoTeste_nan'])).rolling(min_periods=1, window=janela).sum()/janela
    janela = 7
    if (i == 0):
      total = fe
    else:
      total = total + fe
    for i in range(len(fe.columns)):
      fe[fe.columns[i]] = fe[fe.columns[i]].rolling(min_periods=1, window=janela).sum()/janela

    ax = fe.plot.area(rot=90, figsize = (20,8), color = ["blue","gold","black", "red"])
    ax.set_title(estado + fe.index[-1].strftime(' %d/%m/%Y'))
    ax.set_xlabel("Data do teste")
    ax.set_ylabel("Quantidade de testes")
    ax.figure.savefig(caminho + "img/"+ estado + '.png')
